## Welcome

This notebook is a small guide to using with Python to access MarkLogic data. 
It comes with a example MarkLogic project you should install first and the rest of this notebook assumes you have done so. The project creates:

a) A database with a REST endpoint on port 8334
b) A small set of JSON sayings by famous people (in JSON) and some photos of them.

We're going to create a small interface that takes a category of saying, e.g. "inspirational" and returns a list of people who said them.

#### If you haven't already done so, deploy the application to your local server by executing the cell below:

In [ ]:
!gradle mlDeploy

#### Once that's done - load the data into the application

In [ ]:
!gradle importData

The main data consists of a set of sayings records in JSON as below:
```
{
"saying": {
    "quote": "Buy the best, and hope it's good enough.", 
    "person": "Albert Etter", 
    "born": "Siloam Springs, Benton, Arkansas, USA", 
    "dob": "1876-09-20", 
    "occupation": "train engineer", 
    "category": ["practical","fun"]
    }
}
```
First goal then is to get a list of all the unique categories. For that we're usinging the 'values' REST endpoint. To make that work, we have already:

- Configured an index on category in the configuration of the [Content database](files/src/main/ml-config/databases/content-database.json).
- Created an [Options](files/src/main/ml-modules/options/sayings-options.xml) file to configure the REST call

With those in place, we can call the REST endpoint. We'll use the standard Requests library for this. For these sorts of things, it's often easier to work to a DataFrame than raw JSON, so we'll also transform it into that.


In [7]:
import requests
from requests.auth import HTTPDigestAuth
import pandas
import json

In [8]:
def ml_values(attr):
    url = 'http://localhost:8334/v1/values/%s?options=sayings-options' % (attr)
    headers = {'Accept': 'application/json'}
    result = requests.get(url, headers=headers,auth=HTTPDigestAuth('admin','admin'))
    status = str(result.status_code)
    df = None 
    if status == '200':
        v_raw = result.content.decode('utf-8')
        v_json=json.loads(v_raw)['values-response']['distinct-value']
        df = pandas.DataFrame.from_dict(v_json).rename(index=str, columns={"_value": attr})
    else:
        print(status)
        error = result.json()["errorResponse"]["message"]
        print(error)
    return df

In [9]:
ml_values('categories')

,categories,frequency
0,fun,13
1,inspirational,23
2,political,10
3,practical,12
4,pratical,1
5,religious,1
6,spiritual,9
